In [1]:
using MAT
using DelimitedFiles
using Plots
using CausalityTools
using LIBSVM
using ScikitLearn
using LaTeXStrings
# using Metrics
using Statistics
using PyCall
np = pyimport("numpy")
using PyPlot
const plt = PyPlot;
using Distributions, Random
using Printf
using NPZ

In [2]:
if isdir("model_mutual_info")
else
    mkdir("model_mutual_info");
end

In [3]:
Random.seed!(1234);
d = Normal(0,0.0001)
td = truncated(d, -Inf, Inf)

measure = MIShannon(; base = 2)
stim = npzread("rawData/stim.npy");
tf=1000
T=50
y = StateSpaceSet(stim[1:tf-T])

layers = ["in","E1","B","E2","out"]
# layer = "out"

numseeds=25
for layer in layers
    println(layer)
    spks = npzread("rawData/seed0/"*string(layer)*"_spks.npy");
    spks = Int.(spks) # timesteps X neuron
    num_nrn = size(spks)[2]
    println(num_nrn)
    all_cmis = zeros(numseeds,num_nrn)
    
    for seed in 0:numseeds-1
        spks = npzread("rawData/seed"*string(seed)*"/"*string(layer)*"_spks.npy");
        spks = Int.(spks) # timesteps X neuron
        num_nrn = size(spks)[2]
        T = 50
        tf = size(spks)[1]
        counts = Int.(zeros(tf-T,num_nrn))
    
        for t in 1:tf-T
            sub = spks[t:t+T,:]
            counts[t,:] = dropdims(sum(sub, dims=1), dims=1)
        end
    
        for nrn in 1:num_nrn
            # noise = rand(d,size(counts[:,nrn]))
            noise = 0.0001.*randn(size(counts[:,nrn]))
            X = StateSpaceSet(counts[:,nrn] .+ noise)
            mi = mutualinfo(measure, KSG2(k = 3), X, y)
            all_cmis[seed+1,nrn] = mi*(mi>0)
        end
    end
    npzwrite("model_mutual_info/countMI_"*string(layer)*".npy", all_cmis)
end

# fig, ax = plt.subplots()
# box = ax.boxplot(all_cmis)
# plt.ylim(-0.1,1.8)
# plt.show()

in
48
E1
1000
B
10
E2
100
out
10


In [4]:
Random.seed!(1234);
measure = MIShannon(; base = 2)

stim = npzread("rawData/stim.npy");

layers = ["in","E1","B","E2","out"]
num_nrns_vec = [48,1000,10,100,10] # number of neurons in each layer
numLayers = length(layers)

tf=1000
T=50

for li in 1:numLayers
    layer = layers[li]
    num_nrns = num_nrns_vec[li]
    println(layer)
    numseeds=25
    num_nrns=10
    all_tmis=zeros(numseeds,num_nrns)
    
    for seed in 0:numseeds-1
        out_spks = npzread("rawData/seed"*string(seed)*"/"*layer*"_spks.npy");
        spks = Int.(out_spks) # timesteps X neuron
        for nrn_idx in 1:num_nrns
            spks_nrn1 = spks[:,nrn_idx]
            counts = Int.(zeros(tf-T))
            for t in 1:tf-T
                sub = spks_nrn1[t:t+T]
                counts[t] = sum(sub)
            end
    
            # get spike count probability distribution "pcount"
            histstuff=plt.hist(counts,bins=np.arange(0.5,maximum(counts)+1,1)) # exclude spike count = 0
            plt.close()
            num_obs = Int.(histstuff[1]) # number of observations of each spike count
            bin_edges = histstuff[2]
            pcount = num_obs./sum(num_obs)
    
            # create dictionary to store spike timing distributions conditioned on spike count
            spk_timing_dict = Dict()
            y_dict = Dict() # for storing stimuli conditioned on spike count
            for spk_count in 1:maximum(counts)
                key = "spk_count_"*string(spk_count)
                counter_key = "counter_"*string(spk_count)
                spk_timing_dict[counter_key] = 1
                value = zeros(num_obs[spk_count],spk_count)
                spk_timing_dict[key] = value
                yvalue = zeros(num_obs[spk_count])
                y_dict[key] = yvalue
            end
    
    
            for t in 1:tf-T
                sub = spks_nrn1[t:t+T]
                spk_count = sum(sub)
                y_this_ws = stim[t] # stim at this wingstroke
                if spk_count < 1
                    continue
                else
                    strokes_nonans=[]
                    for i in 1:length(sub)
                        if sub[i] == 1
                            push!(strokes_nonans,i)
                        end
                    end
                    key = "spk_count_"*string(spk_count)
                    counter_key = "counter_"*string(spk_count)
                    idx = spk_timing_dict[counter_key]
                    spk_timing_dict[key][idx,:] = strokes_nonans
                    y_dict[key][idx] = y_this_ws
                    spk_timing_dict[counter_key] += 1
                end
            end
    
            # compute timing mutual information
            tmi=0
            for spk_count in 1:maximum(counts)
                key = "spk_count_"*string(spk_count)
                Stpre = spk_timing_dict[key]
                if size(Stpre)[1] < size(Stpre)[2]
                    continue
                else
                    try
                        noise = 0.0001*randn(size(Stpre))
                        St = StateSpaceSet(Stpre .+ noise)
                        tau = StateSpaceSet(y_dict[key])
                        mi = mutualinfo(measure, KSG2(k = 3), St, tau)
                        tmi += pcount[spk_count]*mi*(mi>0)
                    catch
                        println("BoundsError")
                    end
                end
            end
    
    #         print(string(tmi)*" bits")
            all_tmis[seed+1,nrn_idx] = tmi
        end
    end
    npzwrite("model_mutual_info/timingMI_"*layer*".npy", all_tmis)
end

in
E1
BoundsError
B
BoundsError
E2
BoundsError
out
BoundsError
